In [1]:
import logging.config
import math
import random
import gym
import numpy as np
import pkg_resources
from gym import spaces

In [2]:
from astropy import units as u
from poliastro.bodies import Earth
from poliastro.twobody import Orbit
from poliastro.plotting import OrbitPlotter2D
from poliastro.maneuver import Maneuver
from IPython.display import clear_output
from time import sleep
from prettytable import PrettyTable

In [4]:
""" Goal spaceship data"""
r = [-6045, -3490, 2500] << u.km
v = [-3.457, 6.618, 2.533] << u.km / u.s
nostromo = Orbit.from_vectors(Earth, r, v)

class Poliastro_env(gym.Env):
    

    def __init__(self,r_1= [-6045, -3490, 2500]<< u.km,
                      v_1= [-3.457, 6.618, 2.533]<< u.km/u.s):
        
        # Initial orbit
        self.r_1 = r_1
        self.v_1 = v_1
        self.orbit = Orbit.from_vectors(Earth, r_1, v_1)
        
        #Action space 
        self.action_space = spaces.Box(low=np.array([-5, -5, -5]),
                                       high=np.array([5,5,5]),
                                       dtype=np.float16)
        
        #observation space
        self.observation_space = spaces.Box(low=-10000, high=10000, shape=(1,1,1),
                                       dtype=np.float16)
        
    def _get_obs(self):
        
        return {"agent": self.orbit.r.value, "target": nostromo.r.value}
    
    def step(self,action):
           
        """Take the action and make time pass"""
        dv = action << (u.m / u.s)
        self.orbit = self.orbit.apply_maneuver(Maneuver.impulse(dv))
        self.orbit=self.orbit.propagate(10<<u.min)
        
        """Get reward """
        reward=self.get_reward()
        
        """Get observation"""
        observation = self._get_obs()
        
        """If crash, end the task"""
        done=True if self.ground_check() or self.too_far() or reward>=10 else False
         
        return reward, done, observation
    
    def get_reward(self):
        
        dist=self.check_distance()
        
        if dist<100:
            return int(100-dist)
        return 0
              
    def ground_check(self): 
        
        perigee=self.orbit.a*(1-self.orbit.ecc) 
    
        if perigee <= 6371<< u.km:
            return True    
        return False
    
    def too_far(self):
        
        return True if self.orbit.a>16000<< u.km else False
    
    def check_distance(self):
        
        """ Get the difference between both orbits
        need a formula for it"""
        
        a=self.orbit.a.value
        b=nostromo.a.value
        e=self.orbit.ecc.value
        d=nostromo.ecc.value
        
        diff=0
        for x in np.linspace(0, 6, 20):
            r_1=a*(1-e**2)/(1+e*np.cos(x))
            r_2=b*(1-d**2)/(1+d*np.cos(x))
            diff+=abs(r_1-r_2)
           
        return diff

    def reset(self):
        self.orbit=Orbit.from_vectors(Earth, self.r_1, self.v_1)
        return None

    def render(self):
        pass

# Env Call

In [5]:
env=Poliastro_env()

/opt/anaconda3/envs/rl/lib/python3.10/site-packages/gym/spaces/box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float16
  logger.warn(


In [6]:
env._get_obs()

{'agent': array([-6045., -3490.,  2500.]),
 'target': array([-6045., -3490.,  2500.])}

In [7]:
episodes=50

for episode in range(1,episodes+1):
    
    env.reset()
    done=False
    score=0
    steps=0
    
    while not done:
        
        #env.render()
        action=env.action_space.sample()
        reward, done, observation = env.step(action)
        score+=reward
        steps+=1
        if steps>12000:
            break
        
    print("Episode:{}  Score:{}%  Steps: {}".format(episode, score, steps))


Episode:1  Score:22%  Steps: 3166
Episode:2  Score:67%  Steps: 2
Episode:3  Score:64%  Steps: 3
Episode:4  Score:31%  Steps: 84
Episode:5  Score:58%  Steps: 1
Episode:6  Score:73%  Steps: 1
Episode:7  Score:0%  Steps: 12001
Episode:8  Score:66%  Steps: 1
Episode:9  Score:48%  Steps: 2
Episode:10  Score:69%  Steps: 162
Episode:11  Score:14%  Steps: 1
Episode:12  Score:22%  Steps: 260
Episode:13  Score:0%  Steps: 12001
Episode:14  Score:29%  Steps: 329
Episode:15  Score:34%  Steps: 1
Episode:16  Score:0%  Steps: 12001
Episode:17  Score:11%  Steps: 46
Episode:18  Score:60%  Steps: 1
Episode:19  Score:21%  Steps: 831
Episode:20  Score:76%  Steps: 1
Episode:21  Score:87%  Steps: 2
Episode:22  Score:40%  Steps: 1068
Episode:23  Score:43%  Steps: 534
Episode:24  Score:12%  Steps: 2
Episode:25  Score:12%  Steps: 4
Episode:26  Score:0%  Steps: 12001
Episode:27  Score:17%  Steps: 1
Episode:28  Score:64%  Steps: 1
Episode:29  Score:89%  Steps: 2
Episode:30  Score:13%  Steps: 2
Episode:31  Score:2